<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 7.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE',
       'BANKBARODA', 'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA',
       'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB',
       'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'IBREALEST',
       'ICICIBANK', 'ICICIPRULI', 'INDIANB', 'INDIGO', 'INDUSINDBK',
       'INDUSTOWER', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS',
       'MUTHOOTFIN', 'NAM-INDIA', 'NATIONALUM', 'NAUKRI', 'PFIZER',
       'PIDILITIND', 'PVR', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'VOLTAS', 'WHIRLPOOL',
       'WIPRO', 'ZEEL'], dtype=object)

In [4]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    60
1    13
Name: InPortfolio, dtype: int64

In [5]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    8
1    5
Name: CCIFlag, dtype: int64

In [6]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [7]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [8]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Stage'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Stage'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [9]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-01-05 00:00:00+05:30,17992.0,^NSEI,3.56,0.70,0.27,18812.0,15294.0,76.70,23.30,41.0,5.0
2023-01-12 00:00:00+05:30,17858.0,^NSEI,2.51,0.59,-0.01,18812.0,15294.0,72.89,27.11,40.0,5.0
2023-01-19 00:00:00+05:30,18108.0,^NSEI,3.53,0.66,-0.04,18812.0,15294.0,79.98,20.02,50.0,4.0
2023-01-27 00:00:00+05:30,17604.0,^NSEI,0.17,0.87,-0.23,18812.0,15294.0,65.67,34.33,35.0,7.0
2023-02-03 00:00:00+05:30,17854.0,^NSEI,1.22,0.84,-0.34,18812.0,15294.0,72.77,27.23,47.0,5.0


In [10]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [11]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg', 'EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'EstmtFlag','DuraFlag','Stage']

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))

cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Profit%FC', 'Profit%52W', 'Profit%Avg', 'EstmtFlag','DuraFlag','Stage']
tmp_df = tmp_df[cols]
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 13


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
26,VOLTAS.NS,813.0,-8.96,-1.34,-0.12,1315.0,745.0,11.93,88.07,54.0,976.0,821.0,43.4,152.00,19.0,60.0,40.0,0.0,1.0,D
5,ICICIPRULI.NS,418.0,-17.88,-0.59,-0.87,595.0,402.0,8.29,91.71,33.0,594.0,443.0,60.9,79.00,34.0,34.0,34.0,1.0,0.0,D
1,JUBLFOOD.NS,430.0,-23.62,-0.36,-1.76,647.0,430.0,0.00,100.00,18.0,636.0,484.0,87.9,67.50,31.0,34.0,32.0,1.0,1.0,B
38,TATAMOTORS.NS,445.0,4.08,0.47,0.48,507.0,372.0,54.41,45.59,65.0,509.0,392.0,19.2,1000.00,30.0,29.0,30.0,1.0,-1.0,B
12,MOTILALOFS.NS,697.0,-4.06,-0.41,0.14,937.0,654.0,15.35,84.65,48.0,915.0,733.0,19.7,9.65,25.0,28.0,26.0,-1.0,0.0,D
45,TECHM.NS,1011.0,-1.91,0.48,-0.29,1520.0,935.0,13.04,86.96,44.0,1124.0,1048.0,18.7,19.00,7.0,45.0,26.0,-1.0,1.0,B
4,HDFCLIFE.NS,489.0,-12.38,0.00,-0.17,633.0,489.0,-0.07,100.07,17.0,709.0,544.0,89.3,69.50,30.0,16.0,23.0,1.0,1.0,A
28,MUTHOOTFIN.NS,1028.0,-3.07,0.28,-0.28,1443.0,956.0,14.68,85.32,41.0,1227.0,1082.0,12.5,11.00,13.0,33.0,23.0,-1.0,0.0,B
14,BATAINDIA.NS,1530.0,-12.95,-0.40,-1.04,1971.0,1496.0,7.05,92.95,39.0,1998.0,1632.0,62.3,63.60,22.0,21.0,22.0,1.0,1.0,D
37,BERGEPAINT.NS,558.0,-8.48,-0.16,-1.03,735.0,540.0,9.38,90.62,47.0,637.0,570.0,72.9,60.60,12.0,29.0,20.0,1.0,1.0,B


In [12]:
# wait for max profit

tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
1,JUBLFOOD.NS,430.0,-23.62,-0.36,-1.76,647.0,430.0,0.00,100.00,18.0,636.0,484.0,87.9,67.5,31.0,34.0,32.0,1.0,1.0,B
4,HDFCLIFE.NS,489.0,-12.38,0.00,-0.17,633.0,489.0,-0.07,100.07,17.0,709.0,544.0,89.3,69.5,30.0,16.0,23.0,1.0,1.0,A
14,BATAINDIA.NS,1530.0,-12.95,-0.40,-1.04,1971.0,1496.0,7.05,92.95,39.0,1998.0,1632.0,62.3,63.6,22.0,21.0,22.0,1.0,1.0,D
37,BERGEPAINT.NS,558.0,-8.48,-0.16,-1.03,735.0,540.0,9.38,90.62,47.0,637.0,570.0,72.9,60.6,12.0,29.0,20.0,1.0,1.0,B
52,INFY.NS,1599.0,6.13,0.53,-0.19,1866.0,1351.0,48.23,51.77,65.0,1742.0,1628.0,23.0,28.2,7.0,15.0,11.0,1.0,1.0,A


In [13]:
# keep a watch
tmp_df[((tmp_df['EstmtFlag'] == 0) & (tmp_df['DuraFlag'] != -1)) | ((tmp_df['DuraFlag'] == 0) & (tmp_df['EstmtFlag'] != -1))].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
26,VOLTAS.NS,813.0,-8.96,-1.34,-0.12,1315.0,745.0,11.93,88.07,54.0,976.0,821.0,43.4,152.0,19.0,60.0,40.0,0.0,1.0,D
5,ICICIPRULI.NS,418.0,-17.88,-0.59,-0.87,595.0,402.0,8.29,91.71,33.0,594.0,443.0,60.9,79.0,34.0,34.0,34.0,1.0,0.0,D
15,NAUKRI.NS,3657.0,-8.25,-0.10,-0.52,4977.0,3363.0,18.22,81.78,41.0,4684.0,4154.0,78.8,15.8,13.0,20.0,16.0,1.0,0.0,B


In [14]:
# sell as soon as you can
tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
38,TATAMOTORS.NS,445.0,4.08,0.47,0.48,507.0,372.0,54.41,45.59,65.0,509.0,392.0,19.2,1000.00,30.0,29.0,30.0,1.0,-1.0,B
12,MOTILALOFS.NS,697.0,-4.06,-0.41,0.14,937.0,654.0,15.35,84.65,48.0,915.0,733.0,19.7,9.65,25.0,28.0,26.0,-1.0,0.0,D
45,TECHM.NS,1011.0,-1.91,0.48,-0.29,1520.0,935.0,13.04,86.96,44.0,1124.0,1048.0,18.7,19.00,7.0,45.0,26.0,-1.0,1.0,B
28,MUTHOOTFIN.NS,1028.0,-3.07,0.28,-0.28,1443.0,956.0,14.68,85.32,41.0,1227.0,1082.0,12.5,11.00,13.0,33.0,23.0,-1.0,0.0,B
22,DIVISLAB.NS,2884.0,-18.52,-0.54,-0.18,4530.0,2884.0,0.02,99.98,23.0,3558.0,3671.0,45.2,32.00,-3.0,23.0,10.0,-1.0,1.0,B


In [15]:
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg','EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]

In [16]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 4


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
1,JUBLFOOD.NS,430.0,-23.62,-0.36,-1.76,647.0,430.0,0.00,100.00,18.0,636.0,87.9,67.5,48.0,50.0,49.0,1.0,1.0,B
37,BERGEPAINT.NS,558.0,-8.48,-0.16,-1.03,735.0,540.0,9.38,90.62,47.0,637.0,72.9,60.6,14.0,32.0,23.0,1.0,1.0,B
34,ASIANPAINT.NS,2760.0,-12.40,0.16,-1.23,3566.0,2576.0,18.63,81.37,36.0,3180.0,73.4,69.3,15.0,29.0,22.0,1.0,1.0,T
58,LTIM.NS,4502.0,-0.08,0.58,-0.73,6333.0,3761.0,28.82,71.18,58.0,4663.0,26.6,38.6,4.0,41.0,22.0,1.0,0.0,O


In [17]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 9


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
3,WHIRLPOOL.NS,1261.0,-21.45,-0.44,-1.15,1839.0,1260.0,0.14,99.86,17.0,1854.0,72.2,66.7,47.0,46.0,46.0,1.0,0.0,O
18,BAJAJFINSV.NS,1337.0,-13.77,0.71,-2.21,1805.0,1093.0,34.33,65.67,40.0,1691.0,39.2,35.6,26.0,35.0,30.0,1.0,0.0,D
20,BAJFINANCE.NS,6014.0,-10.83,0.37,-1.35,7720.0,5265.0,30.52,69.48,48.0,7391.0,50.8,33.8,23.0,28.0,26.0,1.0,1.0,T
39,TATAPOWER.NS,205.0,-7.52,0.00,-0.94,287.0,195.0,11.20,88.80,46.0,232.0,20.4,25.8,13.0,40.0,26.0,1.0,0.0,O
11,SBIN.NS,544.0,-2.65,1.25,-1.18,626.0,434.0,57.40,42.60,37.0,716.0,13.2,11.7,32.0,15.0,24.0,1.0,0.0,O
35,DMART.NS,3470.0,-15.21,0.05,-1.08,4577.0,3231.0,17.78,82.22,33.0,4006.0,138.7,95.8,15.0,32.0,24.0,1.0,1.0,T
21,RELIANCE.NS,2329.0,-7.58,-0.40,-0.91,2811.0,2229.0,17.18,82.82,28.0,2854.0,25.3,24.8,23.0,21.0,22.0,1.0,0.0,O
34,ASIANPAINT.NS,2760.0,-12.40,0.16,-1.23,3566.0,2576.0,18.63,81.37,36.0,3180.0,73.4,69.3,15.0,29.0,22.0,1.0,1.0,T
58,LTIM.NS,4502.0,-0.08,0.58,-0.73,6333.0,3761.0,28.82,71.18,58.0,4663.0,26.6,38.6,4.0,41.0,22.0,1.0,0.0,O


In [18]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 79]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 6


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
3,WHIRLPOOL.NS,1261.0,-21.45,-0.44,-1.15,1839.0,1260.0,0.14,99.86,17.0,1854.0,72.2,66.7,47.0,46.0,46.0,1.0,0.0,O
39,TATAPOWER.NS,205.0,-7.52,0.00,-0.94,287.0,195.0,11.20,88.80,46.0,232.0,20.4,25.8,13.0,40.0,26.0,1.0,0.0,O
21,RELIANCE.NS,2329.0,-7.58,-0.40,-0.91,2811.0,2229.0,17.18,82.82,28.0,2854.0,25.3,24.8,23.0,21.0,22.0,1.0,0.0,O
35,DMART.NS,3470.0,-15.21,0.05,-1.08,4577.0,3231.0,17.78,82.22,33.0,4006.0,138.7,95.8,15.0,32.0,24.0,1.0,1.0,T
34,ASIANPAINT.NS,2760.0,-12.40,0.16,-1.23,3566.0,2576.0,18.63,81.37,36.0,3180.0,73.4,69.3,15.0,29.0,22.0,1.0,1.0,T
71,IRCTC.NS,627.0,-7.59,0.29,-1.36,857.0,570.0,20.02,79.98,41.0,571.0,NaN,1000.0,-9.0,37.0,14.0,1.0,1.0,O


In [19]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 11


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
55,M&M.NS,1388.0,10.25,1.67,1.01,1388.0,684.0,100.00,0.00,68.0,1475.0,20.5,20.6,6.0,0.0,3.0,1.0,1.0,O
72,BRITANNIA.NS,4629.0,16.42,1.79,0.73,4629.0,2995.0,99.99,0.01,70.0,3935.0,NaN,60.9,-15.0,0.0,-8.0,1.0,0.0,A
70,ITC.NS,381.0,15.70,1.82,0.59,381.0,204.0,99.80,0.20,83.0,350.0,NaN,27.4,-8.0,0.0,-4.0,1.0,0.0,O
66,HCLTECH.NS,1146.0,15.19,1.21,0.37,1150.0,857.0,98.69,1.31,68.0,1117.0,16.3,21.5,-3.0,0.0,-2.0,1.0,1.0,A
68,INDIANB.NS,295.0,28.17,3.91,0.70,305.0,127.0,94.27,5.73,54.0,284.0,7.4,7.2,-4.0,3.0,-0.0,1.0,1.0,O
53,HINDUNILVR.NS,2649.0,2.74,0.97,0.34,2722.0,1897.0,91.12,8.88,55.0,2832.0,NaN,63.0,7.0,3.0,5.0,1.0,1.0,A
50,HDFCBANK.NS,1659.0,9.49,1.32,0.12,1696.0,1281.0,91.04,8.96,57.0,1813.0,26.0,21.1,9.0,2.0,6.0,1.0,0.0,A
61,SUNPHARMA.NS,1028.0,7.43,1.57,0.20,1052.0,791.0,90.86,9.14,51.0,1056.0,NaN,30.6,3.0,2.0,2.0,1.0,1.0,O
49,HDFC.NS,2695.0,8.57,1.41,0.00,2775.0,2053.0,88.87,11.13,54.0,2940.0,23.0,19.6,9.0,3.0,6.0,1.0,0.0,A
36,FEDERALBNK.NS,133.0,8.46,2.44,0.74,140.0,82.0,88.62,11.38,47.0,152.0,11.0,10.0,14.0,5.0,10.0,1.0,0.0,A


In [20]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > -.5) & (tmp_df['Slope%_150'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
21,RELIANCE.NS,2329.0,-7.58,-0.40,-0.91,2811.0,2229.0,17.18,82.82,28.0,2854.0,25.3,24.8,23.0,21.0,22.0,1.0,0.0,O
3,WHIRLPOOL.NS,1261.0,-21.45,-0.44,-1.15,1839.0,1260.0,0.14,99.86,17.0,1854.0,72.2,66.7,47.0,46.0,46.0,1.0,0.0,O


In [21]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [22]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [23]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [24]:
# star stocks performance

star_stocks = ['BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS', 'CANBK.NS',
       'AUBANK.NS', 'TCS.NS', 'INDUSINDBK.NS', 'ICICIBANK.NS', 'WIPRO.NS',
       'AXISBANK.NS', 'HCLTECH.NS', 'M&M.NS', 'HDFCBANK.NS', 'TECHM.NS',
       'INDIANB.NS', 'INFY.NS', 'KOTAKBANK.NS', 'HINDUNILVR.NS',
       'DABUR.NS', 'BHARTIARTL.NS', 'JUBLFOOD.NS', 'TATAPOWER.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
1,JUBLFOOD.NS,430.0,-23.62,-0.36,-1.76,647.0,430.0,0.00,100.00,18.0,636.0,87.9,67.50,48.0,50.0,49.0,1.0,1.0,B
20,BAJFINANCE.NS,6014.0,-10.83,0.37,-1.35,7720.0,5265.0,30.52,69.48,48.0,7391.0,50.8,33.80,23.0,28.0,26.0,1.0,1.0,T
39,TATAPOWER.NS,205.0,-7.52,0.00,-0.94,287.0,195.0,11.20,88.80,46.0,232.0,20.4,25.80,13.0,40.0,26.0,1.0,0.0,O
17,INDUSINDBK.NS,1104.0,-1.29,1.97,-0.67,1264.0,771.0,67.45,32.55,39.0,1402.0,22.8,12.60,27.0,14.0,20.0,1.0,1.0,O
16,ICICIBANK.NS,864.0,-0.60,1.15,-0.90,953.0,650.0,70.56,29.44,49.0,1094.0,24.2,18.90,27.0,10.0,18.0,1.0,1.0,O
29,EICHERMOT.NS,3317.0,-0.55,0.99,-0.37,3850.0,2241.0,66.86,33.14,58.0,3916.0,40.1,38.80,18.0,16.0,17.0,1.0,1.0,T
25,AXISBANK.NS,883.0,7.37,2.07,0.00,962.0,625.0,76.44,23.56,41.0,1060.0,31.3,13.20,20.0,9.0,14.0,1.0,0.0,A
52,INFY.NS,1599.0,6.13,0.53,-0.19,1866.0,1351.0,48.23,51.77,65.0,1742.0,23.0,28.20,9.0,17.0,13.0,1.0,1.0,A
51,CANBK.NS,296.0,10.22,2.97,-0.95,337.0,175.0,75.00,25.00,43.0,322.0,8.8,5.55,9.0,14.0,12.0,1.0,1.0,T
44,AUBANK.NS,629.0,0.83,0.00,0.00,717.0,532.0,52.51,47.49,50.0,698.0,NaN,123.00,11.0,14.0,12.0,1.0,0.0,D


In [25]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] == -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
7,IBREALEST.NS,68.0,-12.08,0.00,-2.53,146.0,60.0,8.95,91.05,32.0,95.0,11.3,1000.00,40.0,115.0,78.0,-1.0,-1.0,B
2,INDUSTOWER.NS,144.0,-26.71,-1.53,-3.26,245.0,137.0,6.16,93.84,28.0,213.0,NaN,14.40,48.0,70.0,59.0,-1.0,1.0,O
0,GODREJIND.NS,432.0,-3.02,0.00,0.45,608.0,405.0,13.08,86.92,46.0,638.0,39.5,18.80,48.0,41.0,44.0,0.0,1.0,B
8,NAM-INDIA.NS,231.0,-14.91,-0.37,-0.78,340.0,231.0,-0.37,100.37,32.0,322.0,29.9,20.60,39.0,47.0,43.0,-1.0,1.0,O
32,NATIONALUM.NS,78.0,4.60,1.33,1.27,126.0,66.0,20.75,79.25,40.0,90.0,10.3,5.72,15.0,62.0,38.0,0.0,1.0,O
10,ZEEL.NS,221.0,-11.41,0.40,-1.24,299.0,206.0,15.70,84.30,38.0,303.0,24.6,26.10,37.0,35.0,36.0,-1.0,1.0,O
6,SBILIFE.NS,1138.0,-8.81,0.72,-0.56,1332.0,1023.0,37.23,62.77,30.0,1607.0,62.1,1000.00,41.0,17.0,29.0,0.0,1.0,O
9,PFIZER.NS,3873.0,-9.22,-0.12,-1.25,4599.0,3821.0,6.64,93.36,24.0,5297.0,36.5,116.00,37.0,19.0,28.0,0.0,1.0,O
19,PVR.NS,1692.0,-6.98,-0.44,-0.46,2193.0,1498.0,27.91,72.09,49.0,2106.0,51.2,1000.00,24.0,30.0,27.0,1.0,-1.0,O
13,EMAMILTD.NS,420.0,-8.31,0.44,0.00,517.0,397.0,19.13,80.87,43.0,550.0,49.6,22.30,31.0,23.0,27.0,-1.0,1.0,T
